# 20x20 Random Maze Generator using Depth-First-Search

In [1]:
#Alex Lewandowski
# 02-2019

# arcade.download() # for downloading packages
import arcade
import collections

N = 700 #Wwindow side

class Vertex:
    def __init__(self, x, y, wall_N, wall_S, wall_E, wall_W): 
        self.x = x
        self.y = y
        self.state = 0
        self.wall_N = Wall('N', wall_N, self)
        self.wall_S = Wall('S', wall_S, self)
        self.wall_E = Wall('E', wall_E, self)
        self.wall_W = Wall('W', wall_W, self)
        self.walls = [self.wall_N, self.wall_S, self.wall_E, self.wall_W]
        self.type = 0
        self.visited = False
        self.seen = False
        self.h = 0
        self.f = 0
        self.g = 0

    def draw(self):
        if (self.state != 0):
            arcade.draw_circle_filled(self.x, self.y, N/80, self.state)

class Wall:
    def __init__(self, wallDirection, on, vertex):
        self.on = True
        self.wallDirection = wallDirection
        if(wallDirection == 'N'):
            self.x1 = vertex.x - (N/40)
            self.y1 = vertex.y + (N/40)
            self.x2 = vertex.x + (N/40)
            self.y2 = vertex.y + (N/40)
        elif(wallDirection == 'S'):
            self.x1 = vertex.x - (N/40)
            self.y1 = vertex.y - (N/40)
            self.x2 = vertex.x + (N/40)
            self.y2 = vertex.y - (N/40)
        elif(wallDirection == 'E'):
            self.x1 = vertex.x + (N/40)
            self.y1 = vertex.y - (N/40)
            self.x2 = vertex.x + (N/40)
            self.y2 = vertex.y + (N/40)
        elif(wallDirection == 'W'):
            self.x1 = vertex.x - (N/40)
            self.y1 = vertex.y - (N/40)
            self.x2 = vertex.x - (N/40)
            self.y2 = vertex.y + (N/40)



    def draw(self):
        if(self.on):
            arcade.draw_line(self.x1, self.y1, self.x2, self.y2, arcade.color.BLACK, 2.0)

## Mazes Class

### craftRandom()
####   Comment from: https://en.wikipedia.org/wiki/Maze_generation_algorithm.
    
This algorithm is a randomized version of the depth-first search algorithm. Frequently implemented with a stack,       this approach is one of the simplest ways to generate a maze using a computer. Consider the space for a maze being       a large grid of cells (like a large chess board), each cell starting with four walls. Starting from a random cell,       the computer then selects a random neighbouring cell that has not yet been visited. The computer removes the wall       between the two cells and marks the new cell as visited, and adds it to the stack to facilitate backtracking. The       computer continues this process, with a cell that has no unvisited neighbours being considered a dead-end. When at a dead-end it backtracks through the path until it reaches a cell with an unvisited neighbour, continuing the path generation by visiting this new, unvisited cell (creating a new junction). This process continues until every cell has been visited, causing the computer to backtrack all the way back to the beginning cell. We can be sure every cell is visited.

As given above this algorithm involves deep recursion which may cause stack overflow issues on some computer      architectures. The algorithm can be rearranged into a loop by storing backtracking information in the maze itself. This also provides a quick way to display a solution, by starting at any given point and backtracking to the beginning.


### Pseudocode used to write craftRandom()
1.  procedure DFS-iterative(G,v):
2.      let S be a stack
3.      S.push(v)
4.      while S is not empty
5.          v = S.pop()
6.          if v is not labeled as discovered:
7.              label v as discovered
8.              for all edges from v to w in G.adjacentEdges(v) do 
9.                  S.push(w)
 

In [2]:
import random


N = 700 #WINDOW_WIDTH

class Mazes:
    def __init__(self, myMaze):
        self.verticies = self.generateBlank()
        self.begin = self.verticies[0][0]
        self.end = self.verticies[19][19]
        self.cur = self.begin 
        self.myMaze = myMaze
        #self.mazeState = self.myMaze.mazeState

    def generateBlank(self):
        k = N/40
        c = 1
        r = 1
        verticies = []
        while c < 40:
            row = []
            r = 1
            while r < 40:
                row.append(Vertex(c*k, r*k, True, True, True, True))
                r = r + 2
            verticies.append(row)
            c = c + 2
        return verticies

    def updateColor(self):
        self.begin.state = arcade.color.GO_GREEN
        self.end.state = arcade.color.RED

    def validMoves(self, vertexCol, vertexRow): #don't allow visited moves
        moves = []
        if vertexCol > 0:
            if self.verticies[vertexCol-1][vertexRow].visited == False: #add valid West
                moves.append(self.verticies[vertexCol-1][vertexRow])
        if vertexCol < 19:
            if self.verticies[vertexCol+1][vertexRow].visited == False: #add valid East
                moves.append(self.verticies[vertexCol+1][vertexRow])
        if vertexRow > 0:
            if self.verticies[vertexCol][vertexRow-1].visited == False: #add valid South
                moves.append(self.verticies[vertexCol][vertexRow-1])
        if vertexRow < 19:
            if self.verticies[vertexCol][vertexRow+1].visited == False: #add valid North
                moves.append(self.verticies[vertexCol][vertexRow+1])
        return moves

    def visitedNeighbors(self, vertexCol, vertexRow):
        moves = []
        if vertexCol > 0:
            if self.verticies[vertexCol-1][vertexRow].visited == True: #add visited West
                moves.append(self.verticies[vertexCol-1][vertexRow])
        if vertexCol < 19:
            if self.verticies[vertexCol+1][vertexRow].visited == True: #add visited East
                moves.append(self.verticies[vertexCol+1][vertexRow])
        if vertexRow > 0:
            if self.verticies[vertexCol][vertexRow-1].visited == True: #add visitied North
                moves.append(self.verticies[vertexCol][vertexRow-1])
        if vertexRow < 19:
            if self.verticies[vertexCol][vertexRow+1].visited == True: #add visited South
                moves.append(self.verticies[vertexCol][vertexRow+1])
        return moves

    def removeWall(self,vertex1,vertex2):
        vLoc1 = self.getVerticiesLoc(vertex1)
        vLoc2 = self.getVerticiesLoc(vertex2)
        if vLoc1[0] == vLoc2[0]: #then they share a N or S wall
            if vLoc1[1] < vLoc2[1]: 
                vertex1.wall_N.on = False
                vertex2.wall_S.on = False
            elif vLoc1[1] > vLoc2[1]:
                vertex1.wall_S.on = False
                vertex2.wall_N.on = False
        elif vLoc1[1] == vLoc2[1]:
            if vLoc1[0] < vLoc2[0]:
                vertex1.wall_E.on = False
                vertex2.wall_W.on = False
            elif vLoc1[0] > vLoc2[0]:
                vertex1.wall_W.on = False
                vertex2.wall_E.on = False
        else:
            return

    def getVerticiesLoc(self, v):
        c = 1
        r = 1
        for col in range (0, len(self.verticies)):
            for row in range (0, len(self.verticies)):
               # vertX = round(self.verticies[col][row].x)
                if round(self.verticies[col][row].x) == round(v.x) and  round(self.verticies[col][row].y) == round(v.y):
                    return [col, row]
                r = r + 2
            c = c + 2
            
    def craftRandom(self):
            curEndLoc = [0,0]
            stack = [self.begin]       
            while len(stack) > 0:
                v = stack.pop(0)
                v.visited = True
                vLoc = self.getVerticiesLoc(v)  
                valid = self.validMoves(vLoc[0], vLoc[1])
                if len(valid) > 1:
                    move = random.choice(valid)
                    self.removeWall(v, move)
                    move.visited = True

                    for m in valid:
                        if m != move:
                            stack.append(m)
                    stack.insert(0, move)
                elif len(valid) == 1:
                    self.removeWall(v, valid[0])
                    valid[0].visited = True
                    self.updateColor()
                    stack.insert(0, (valid[0]))
                elif len(valid) == 0:
                    while len(stack) > 0 and stack[0].visited != False:
                        stack.pop(0)
                    if len(stack) > 0:
                        sLoc = self.getVerticiesLoc(stack[0])
                        self.removeWall(stack[0], random.choice(self.visitedNeighbors(sLoc[0], sLoc[1])))


In [3]:
class MazeState:
    def __init__(self, maze):
        self.maze = maze
        
    #will be used when writing A* search
    def manhattanDistance(self, vertex1, vertex2):
        return (abs(vertex1.x - vertex2.x) + abs(vertex1.y - vertex2.y))

    def draw(self):
        for col in range (0, len(self.maze)):
            for vertex in self.maze[col]:
                vertex.draw()
                for wall in vertex.walls:
                    wall.draw() 

In [4]:
WINDOW_SIDE = 700

TITLE = "Maze Generator"


####################
##  MyMaze Class  ##
####################
class MyMaze(arcade.Window):
    def __init__(self, width, height, title):
        self.maze = Mazes(self)
        self.maze.craftRandom()
        self.mazeState = MazeState(self.maze.verticies)
        super().__init__(width, height, title)

        self.set_mouse_visible(True)

        arcade.set_background_color(arcade.color.LIGHT_GRAY) 

    def on_draw(self):
            arcade.start_render()
            self.mazeState.draw()        

window = MyMaze(WINDOW_SIDE, WINDOW_SIDE, TITLE)
arcade.run()